In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import LSTM, GRU, Dense, SimpleRNN
from tensorflow.keras.models import Sequential




In [10]:
import yaml

class CustomLoader(yaml.Loader):
    def __init__(self, stream):
        super(CustomLoader, self).__init__(stream)
        self.add_constructor('!obj:pylearn2.train.Train', self.construct_train)
        self.add_constructor('!obj:librnn.pylearn2.datasets.music.MusicSequence', self.construct_generic)
        self.add_constructor('!obj:pylearn2.sandbox.rnn.models.rnn.RNN', self.construct_generic)
        self.add_constructor('!obj:pylearn2.sandbox.rnn.models.rnn.GRU', self.construct_generic)
        self.add_constructor('!obj:librnn.pylearn2.models.mlp.Sigmoid', self.construct_generic)
        self.add_constructor('!obj:pylearn2.sandbox.rnn.space.SequenceDataSpace', self.construct_generic)
        self.add_constructor('!obj:pylearn2.space.VectorSpace', self.construct_generic)
        self.add_constructor('!obj:pylearn2.training_algorithms.sgd.SGD', self.construct_generic)
        self.add_constructor('!obj:librnn.pylearn2.training_algorithms.learning_rule.RMSPropMomentum', self.construct_generic)
        self.add_constructor('!obj:librnn.pylearn2.costs.rnn.RNNCost', self.construct_generic)
        self.add_constructor('!obj:pylearn2.termination_criteria.And', self.construct_generic)
        self.add_constructor('!obj:pylearn2.termination_criteria.EpochCounter', self.construct_generic)
        self.add_constructor('!obj:pylearn2.train_extensions.best_params.MonitorBasedSaveBest', self.construct_generic)
        self.add_constructor('!obj:pylearn2.training_algorithms.sgd.LinearDecayOverEpoch', self.construct_generic)

    def construct_train(self, loader, node):
        # YAML에서 `!obj:pylearn2.train.Train` 태그가 사용될 때 반환될 객체/데이터 정의
        # 여기서는 단순히 dict로 반환
        return loader.construct_mapping(node)

    def construct_generic(self, loader, node):
        # 다른 모든 커스텀 태그는 기본적으로 dict로 변환
        return loader.construct_mapping(node)

def load_data_with_custom_loader(yaml_file_path):
    with open(yaml_file_path, 'r') as file:
        try:
            data = yaml.load(file, Loader=CustomLoader)
        except yaml.YAMLError as e:
            print(f"Error parsing YAML: {e}")
            data = None
    return data

# YAML 파일 경로 설정
gru_yaml_path = 'gru.yaml'
lstm_yaml_path = 'lstm.yaml'

# 사용자 정의 로더로 YAML 파일 로드
gru_data = load_data_with_custom_loader(gru_yaml_path)
lstm_data = load_data_with_custom_loader(lstm_yaml_path)

# 로드된 데이터 출력 (필요 시)
print("GRU Data:", gru_data)
print("LSTM Data:", lstm_data)


GRU Data: {'dataset': {'which_dataset': 'jsb', 'which_set': 'train'}, 'model': {'input_space': {'space': {'dim': 96}}, 'layers': [{'layer_name': 'h_1', 'dim': 48, 'irange': 0.1, 'weight_noise': 1}, {'layer_name': 'y', 'dim': 96, 'irange': 0.1}]}, 'algorithm': {'batch_size': 1, 'learning_rate': 0.001, 'learning_rule': {'init_momentum': 0.9, 'momentum_clipping': 1.0}, 'monitoring_dataset': {'train': {'which_dataset': 'jsb', 'which_set': 'train'}, 'valid': {'which_dataset': 'jsb', 'which_set': 'valid'}}, 'cost': {}, 'termination_criterion': {'criteria': [{'max_epochs': 500}]}}, 'extensions': [{'channel_name': 'valid_objective', 'save_path': 'gru_best.pkl'}, {'start': 1, 'saturate': 500, 'decay_factor': 0.01}], 'save_path': 'gru.pkl', 'save_freq': 10}
LSTM Data: {'dataset': {'which_dataset': 'jsb', 'which_set': 'train'}, 'model': {'input_space': {'space': {'dim': 96}}, 'layers': [{'layer_name': 'h_1', 'dim': 36, 'irange': 0.1, 'weight_noise': 1}, {'layer_name': 'y', 'dim': 96, 'irange': 0.

In [ ]:
# GRU 모델 구성
def build_gru_model(gru_data):
    input_dim = gru_data['model']['input_space']['space']['dim']
    gru_layer_info = gru_data['model']['layers'][0]
    output_dim = gru_data['model']['layers'][1]['dim']
    
    model = Sequential()
    model.add(GRU(gru_layer_info['dim'], input_shape=(None, input_dim), return_sequences=True))
    model.add(Dense(output_dim, activation='linear'))  # 활성화 함수를 'linear'로 설정하여 회귀 모델로 구성

    # SGD 옵티마이저 사용
    optimizer = SGD(learning_rate=gru_data['algorithm']['learning_rate'], momentum=gru_data['algorithm']['learning_rule']['init_momentum'])

    model.compile(optimizer=optimizer, loss=MeanSquaredError(), metrics=[MeanSquaredError()])
    return model

# LSTM 모델 구성
def build_lstm_model(lstm_data):
    input_dim = lstm_data['model']['input_space']['space']['dim']
    lstm_layer_info = lstm_data['model']['layers'][0]
    output_dim = lstm_data['model']['layers'][1]['dim']
    
    model = Sequential()
    model.add(LSTM(lstm_layer_info['dim'], input_shape=(None, input_dim), return_sequences=True))
    model.add(Dense(output_dim, activation='linear'))  # 활성화 함수를 'linear'로 설정하여 회귀 모델로 구성

    # SGD 옵티마이저 사용
    optimizer = SGD(learning_rate=lstm_data['algorithm']['learning_rate'], momentum=lstm_data['algorithm']['learning_rule']['init_momentum'])

    model.compile(optimizer=optimizer, loss=MeanSquaredError(), metrics=[MeanSquaredError()])
    return model

# GRU 및 LSTM 모델 생성
gru_model = build_gru_model(gru_data)
lstm_model = build_lstm_model(lstm_data)

# 모델 구조 확인
gru_model.summary()
lstm_model.summary()

# 데이터 준비 (예시 데이터)
X_train = np.random.rand(1000, 10, 96)  # 예시 데이터
y_train = np.random.rand(1000, 10, 96)  # 예시 데이터
X_valid = np.random.rand(200, 10, 96)   # 예시 데이터
y_valid = np.random.rand(200, 10, 96)   # 예시 데이터

# GRU 모델 훈련
gru_model.fit(X_train, y_train, batch_size=gru_data['algorithm']['batch_size'], epochs=gru_data['algorithm']['termination_criterion']['criteria'][0]['max_epochs'], validation_data=(X_valid, y_valid))

# LSTM 모델 훈련
lstm_model.fit(X_train, y_train, batch_size=lstm_data['algorithm']['batch_size'], epochs=lstm_data['algorithm']['termination_criterion']['criteria'][0]['max_epochs'], validation_data=(X_valid, y_valid))

# 모델 저장
gru_model.save(gru_data['save_path'])
lstm_model.save(lstm_data['save_path'])

# GRU 모델 예측 및 MSE, R² 계산
y_pred_gru = gru_model.predict(X_valid)
mse_gru = np.mean((y_valid - y_pred_gru) ** 2)
r2_gru = r2_score(y_valid.flatten(), y_pred_gru.flatten())

print(f"GRU Model - MSE: {mse_gru}, R²: {r2_gru}")

# LSTM 모델 예측 및 MSE, R² 계산
y_pred_lstm = lstm_model.predict(X_valid)
mse_lstm = np.mean((y_valid - y_pred_lstm) ** 2)
r2_lstm = r2_score(y_valid.flatten(), y_pred_lstm.flatten())

print(f"LSTM Model - MSE: {mse_lstm}, R²: {r2_lstm}")